In [1]:
import pymove as pm
import pandas as pd
import numpy as np
np.seterr(invalid='ignore')
from core.TrajectoryDF import NumPandasTraj as TrajDF
from preprocessing.filters import Filters as filt
from features.spatial_features import SpatialFeatures as spatial
from utilities.conversions import Conversions as con
from preprocessing.interpolation import Interpolate as ip

In [2]:
atlantic = pd.read_csv('./data/atlantic.csv')
atlantic = con.convert_directions_to_degree_lat_lon(atlantic, 'Latitude',"Longitude")
def convert_to_datetime(row):
        this_date = '{}-{}-{}'.format(str(row['Date'])[0:4], str(row['Date'])[4:6], str(row['Date'])[6:])
        this_time = '{:02d}:{:02d}:00'.format(int(row['Time']/100), int(str(row['Time'])[-2:]))
        return '{} {}'.format(this_date, this_time)
atlantic['DateTime'] = atlantic.apply(convert_to_datetime, axis=1)

gulls = pd.read_csv('./data/gulls.csv')

geolife = pd.read_csv('./data/geolife_sample.csv')

In [3]:
%%time

np_atlantic = TrajDF(atlantic,
                         latitude='Latitude',
                         longitude='Longitude',
                         datetime='DateTime',
                         traj_id='ID',
                         rest_of_columns=[])

np_gulls = TrajDF(gulls,
                 latitude='location-lat',
                 longitude='location-long',
                 datetime='timestamp',
                 traj_id='tag-local-identifier',
                 rest_of_columns=[])

np_geolife = TrajDF(geolife,latitude='lat',
                     longitude='lon',
                     datetime='datetime',
                     traj_id='id')


CPU times: user 526 ms, sys: 8.08 ms, total: 534 ms
Wall time: 533 ms


In [4]:
%%time

pm_atlantic = pm.MoveDataFrame(atlantic,
                         latitude='Latitude',
                         longitude='Longitude',
                         datetime='DateTime',
                         traj_id='ID')

pm_gulls = pm.MoveDataFrame(gulls,
                 latitude='location-lat',
                 longitude='location-long',
                 datetime='timestamp',
                 traj_id='tag-local-identifier')

pm_geolife = pm.MoveDataFrame(geolife,
                              latitude='lat',
                              longitude='lon',
                              datetime='datetime',
                              traj_id='id')
print(atlantic.shape)

(49105, 23)
CPU times: user 104 ms, sys: 7.98 ms, total: 112 ms
Wall time: 109 ms


In [5]:
# %%time
#
# linear = ip.interpolate_position(dataframe=np_atlantic,
#                                  ip_type='linear',
#                                  distance_threshold=250000)
# print(linear.shape)
# linear.head()

len(np_geolife)

217653

In [6]:
%%time
np_geolife = spatial.create_distance_between_consecutive_column(np_geolife)
interpolated_geolife = ip.interpolate_position(np_geolife, 30)

CPU times: user 52.9 s, sys: 37.9 ms, total: 52.9 s
Wall time: 52.9 s


In [7]:
print(f"Length of original geolife Dataframe: {len(np_geolife)}")
print(f"Length of interpolated geolife Dataframe: {len(interpolated_geolife)}")

Length of original geolife Dataframe: 217653
Length of interpolated geolife Dataframe: 222460


In [8]:
%%time

np_atlantic = spatial.create_distance_between_consecutive_column(np_atlantic)
interpolated_atlantic = ip.interpolate_position(np_atlantic, 250000)

/home/yjharanwala/anaconda3/envs/PreprocessingLibrary/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


CPU times: user 29.5 s, sys: 485 ms, total: 30 s
Wall time: 31.2 s


In [9]:
print(f"Length of original atlantic Dataframe: {len(np_atlantic)}")
print(f"Length of interpolated atlantic Dataframe: {len(interpolated_atlantic)}")


Length of original atlantic Dataframe: 49105
Length of interpolated atlantic Dataframe: 52837
